In [1]:
import importlib
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [2]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
from cimgraph.models import FeederModel
from cimgraph.models import NodeBreakerModel
from cimgraph.models import BusBranchModel
from cimgraph.tests.test_get_all_edges import test_get_all_edges

In [4]:
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="../cimgraph/tests/test_models/ieee13.xml", cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=rdf, container=feeder, distributed=False)



In [17]:
# Get everything related to buses
network.get_all_edges(cim.Terminal)
network.get_all_edges(cim.ConnectivityNode)
network.get_all_edges(cim.Location)
network.get_all_edges(cim.PositionPoint)
network.get_all_edges(cim.Feeder)
network.get_all_edges(cim.Substation)

In [11]:
import logging
_log = logging.getLogger(__name__)

In [15]:
# Get capacitor data
network.get_all_edges(cim.LinearShuntCompensator)
network.get_all_edges(cim.LinearShuntCompensatorPhase)
network.get_all_edges(cim.RegulatingControl)

41.6666664

In [22]:
# get ditto capacitor info

for element in network.graph[cim.LinearShuntCompensator].values():
    susceptance = float(element.bPerSection)
    conductance = float(element.gPerSection)
    # resistance = 1 / conductance #not in CIM
    susceptance0 = float(element.b0PerSection)
    conductance0 = float(element.g0PerSection)
    # resistance0 = 1 / conductance0
    connecting_element = element.Terminals[0].ConnectivityNode

    if element.RegulatingControl is not None:
        high = element.RegulatingControl #TODO should  be vmax in opendss -- figure out Tom's converter 
        low = element.RegulatingControl #TODO should  be vmax in opendss -- figure out Tom's converter 
        mode = element.RegulatingControl.mode #enum
    delay = float(element.aVRDelay)
    nominal_voltage = float(element.nomU)

    positions = element.Location.PositionPoints

    measuring_element = None #TODO associate with OpenDSS line object meas
    
    pt_ratio = None #missing?
    pt_phase = None
    ct_ratio = None #missing?
    

    connection_type = element.phaseConnection

    container = element.EquipmentContainer
    if container.__class__.__name__ == "Feeder":
        feeder_name = container.name
        substation_name = container.NormalEnergizingSubstation.name
        is_substation = False
    elif container.__class__.__name__ == "Substation":
        feeder_name = None
        substation_name = container.name
        is_substation = True
    else:
        feeder_name = None
        try:
            substation_name = container.Substation.name
        except:
            _log.warning("could not identify feeder or substation")

    for phase in element.ShuntCompensatorPhase:
        switch = phase.sections #TODO: figure out difference between switch and sections
        sections = phase.sections 
        normalsections = phase.normalSections
        var = nominal_voltage * nominal_voltage * float(phase.bPerSection)

        



In [36]:
nominal_voltage * nominal_voltage * float(phase.bPerSection)


99999.99935999999

In [9]:
network.pprint(cim.LinearShuntCompensator, show_empty=True)

{
    "A9DE8829-58CB-4750-B2A2-672846A89753": {
        "mRID": "A9DE8829-58CB-4750-B2A2-672846A89753",
        "aliasName": "",
        "description": "",
        "name": "cap1",
        "Names": "",
        "AssetDatasheet": "",
        "Assets": "",
        "ConfigurationEvent": "",
        "Controls": "",
        "Location": "62A249A4-4BF9-42BB-9E0A-06D132216D9A",
        "Measurements": "",
        "OperatingShare": "",
        "PSRType": "",
        "ReportingGroup": "",
        "aggregate": "",
        "inService": "",
        "networkAnalysisEnabled": "",
        "normallyInService": "",
        "AdditionalEquipmentContainer": "",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "Faults": "",
        "OperationalLimitSet": "",
        "UsagePoints": "",
        "BaseVoltage": "2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
        "SvStatus": "",
        "Terminals": "['5B38070B-E918-4EAA-8BDB-8B3CE4F8A917']",
        "EnergyConnectionProfile": "",
        

In [5]:
# Get everything related to transformers
network.get_all_edges(cim.PowerTransformer)
network.get_all_edges(cim.TransformerTank)
network.get_all_edges(cim.TransformerTankEnd)
network.get_all_edges(cim.TransformerTankInfo)
network.get_all_edges(cim.TransformerEndInfo)
network.get_all_edges(cim.PowerTransformerEnd)
network.get_all_edges(cim.PowerTransformerInfo)
network.get_all_edges(cim.TransformerCoreAdmittance)
network.get_all_edges(cim.TransformerMeshImpedance)
network.get_all_edges(cim.TransformerStarImpedance)
network.get_all_edges(cim.ShortCircuitTest)
network.get_all_edges(cim.NoLoadTest)
network.get_all_edges(cim.RatioTapChanger)
network.get_all_edges(cim.TapChanger)
network.get_all_edges(cim.TapChangerControl)
network.get_all_edges(cim.TapChangerInfo)

In [ ]:
# Power Transformer stuff
#element
for element in network.graph[cim.PowerTransformer]:
    for winding in element.PowerTransformerEnds:
        to_element = winding.Terminals[0].ConnectivityNode #obj
        to_element_connection_index = winding.endNumber #float, 1 is high-side
        from_element = winding.Terminals[1].ConnectivityNode #obj
    
    feeder_name = element.EquipmentContainer.name #str

